<a href="https://colab.research.google.com/github/pseeej/SmartSystemsSoftware/blob/master/BigDataAnalysisInBiz/6%EC%A3%BC%EC%B0%A8_%EB%85%B9%ED%99%94_%EB%B0%8F_%EC%8B%A4%EC%8B%9C%EA%B0%84_%EA%B0%95%EC%9D%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

교재 : SQL로 맛보는 데이터 전처리 분석, 노수영


**실습 참고자료**

https://www.sqlitetutorial.net/
<br>https://www.w3schools.com/sql/default.asp

#실습환경 셋업

## 1) sqlite 업그레이드

In [1]:
!apt-get update
!apt-get upgrade sqlite3

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1,431 kB]
Get:12 http://security.ubuntu.com/ubuntu bionic-security/restricted amd64 Packages 

In [2]:
!curl https://www.sqlite.org/src/tarball/sqlite.tar.gz?r=release | tar xz
%cd sqlite/
!./configure
!make sqlite3.c
%cd /content
!npx degit coleifer/pysqlite3 -f
!cp sqlite/sqlite3.[ch] .
!python setup.py build_static build
!cp build/lib.linux-x86_64-3.7/pysqlite3/_sqlite3.cpython-37m-x86_64-linux-gnu.so \
     /usr/lib/python3.7/lib-dynload/


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 10.9M  100 10.9M    0     0  9769k      0  0:00:01  0:00:01 --:--:-- 9769k
/content/sqlite
checking build system type... x86_64-pc-linux-gnu
checking host system type... x86_64-pc-linux-gnu
checking for gcc... gcc
checking whether the C compiler works... yes
checking for C compiler default output file name... a.out
checking for suffix of executables... 
checking whether we are cross compiling... no
checking for suffix of object files... o
checking whether we are using the GNU C compiler... yes
checking whether gcc accepts -g... yes
checking for gcc option to accept ISO C89... none needed
checking for a sed that does not truncate output... /bin/sed
checking for grep that handles long lines and -e... /bin/grep
checking for egrep... /bin/grep -E
checking for fgrep... /bin/grep -F
checking for ld used by gcc... /usr/bin/ld
checkin

In [1]:
# 메뉴 런타임 > 런타임 다시 시작 후 셀 실행
import pandas as pd
import sqlite3
sqlite3.sqlite_version  # 3.36.0

'3.36.0'

## 2) Instacart 데이터베이스 다운로드

In [2]:
import gdown

url = 'https://drive.google.com/uc?id=17q9zfOWSRVw_H0tQn03xyLvMcy-ynPE6'
output = 'Instacart.db'
gdown.download(url, output, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=17q9zfOWSRVw_H0tQn03xyLvMcy-ynPE6
To: /content/Instacart.db
100%|██████████| 1.07M/1.07M [00:00<00:00, 91.7MB/s]


'Instacart.db'

## 3) Instacart 데이터베이스 연결

In [3]:
conn = sqlite3.connect("Instacart.db")
cur = conn.cursor()

## 4) Instacart 데이터베이스 테이블 조회 테스트

In [4]:
def viewlimit(table, cutlimit):
  sql = "SELECT * FROM " + table + " LIMIT " + cutlimit + ";"
  cur.execute(sql)
  rows = cur.fetchall()
  print(sql)
  for row in rows:
    print(row)

In [5]:
viewlimit("orders","10")

SELECT * FROM orders LIMIT 10;
(2352815, 185282, 'prior', 5, 2, 9, 6)
(1414622, 41949, 'prior', 9, 0, 16, 6)
(2534064, 24631, 'prior', 66, 1, 13, 4)
(338572, 95289, 'prior', 4, 2, 14, 18)
(2578618, 170417, 'prior', 5, 1, 13, 14)
(1756647, 127210, 'prior', 31, 3, 18, 9)
(2636806, 26473, 'prior', 24, 2, 17, 17)
(681993, 89155, 'prior', 17, 1, 15, 4)
(826339, 67750, 'prior', 7, 3, 14, 17)
(57285, 89307, 'prior', 48, 1, 22, 5)


## 5) Pandas read_sql 함수로 데이터 조회 및 결과 확인

In [6]:
window_sql = f"""
select *
  from orders
"""
df = pd.read_sql(window_sql, conn, index_col=None)
df.head(10)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2352815,185282,prior,5,2,9,6
1,1414622,41949,prior,9,0,16,6
2,2534064,24631,prior,66,1,13,4
3,338572,95289,prior,4,2,14,18
4,2578618,170417,prior,5,1,13,14
5,1756647,127210,prior,31,3,18,9
6,2636806,26473,prior,24,2,17,17
7,681993,89155,prior,17,1,15,4
8,826339,67750,prior,7,3,14,17
9,57285,89307,prior,48,1,22,5


In [7]:
#db조회 결과 다운로드
df.to_csv('/content/tmp.csv')

## 6) 구글 드라이브 마운트 및 파일 저장

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


아래 코드 실행 전 내 드라이브에 result 폴더를 생성하고 해당 폴더 안에 6w 폴더를 생성함

In [9]:
def sql_result(sql,file):
  print(sql)
  df = pd.read_sql(sql, conn, index_col=None)
  print(df.head(10))
  file_name = '/content/drive/MyDrive/result/6w/' + file + '.csv' #csv파일을 MyDrive/result/6w 폴더에 저장
  print(file_name)
  df.to_csv(file_name, index=False)

In [10]:
sql = f"""
select *
  from orders
 limit 10
"""
file = 'result_6w_test'

In [11]:
sql_result(sql,file)


select *
  from orders
 limit 10

   order_id  user_id  ... order_hour_of_day  days_since_prior_order
0   2352815   185282  ...                 9                       6
1   1414622    41949  ...                16                       6
2   2534064    24631  ...                13                       4
3    338572    95289  ...                14                      18
4   2578618   170417  ...                13                      14
5   1756647   127210  ...                18                       9
6   2636806    26473  ...                17                      17
7    681993    89155  ...                15                       4
8    826339    67750  ...                14                      17
9     57285    89307  ...                22                       5

[10 rows x 7 columns]
/content/drive/MyDrive/result/6w/result_6w_test.csv


# 1.데이터 생성

데이터 다운로드 URL
https://github.com/billyrohh/instacart_dataset

데이터베이스 소개
https://www.kaggle.com/c/instacart-market-basket-analysis/data

처음 접하는 데이터가 있다면 항상 각 테이블이 무엇을 의미하는지, 어떤 컬럼이 있는 확인하고 테이블 간의 관계를 파악해야 한다.


<center>

|테이블명|내용|
|-----|-----|
|aisles|상품카테고리정보1|
|departments|상품카테고리정보2|
|order_products__prior|각 주문 번호의 상세 구매 내역|
|orders|주문 대표 정보|
|products|상품정보|
</center>

[aisles]
*   aisle_id
*   aisle

[departments]
*   department_id
*   department

[order_products__prior]
*   order_id
*   product_id
*   add_to_cart_order
*   reordered

[orders]
*   order_id
*   user_id
*   eval_set
*   order_number
*   order_dow
*   order_hour_of_day
*   days_since_prior

[products]
*   product_id
*   product_name
*   aisle_id
*   department_id

In [12]:
#테이블 리스트 확인
sql = f"""
SELECT name FROM sqlite_master WHERE type='table';
"""
df = pd.read_sql(sql, conn, index_col=None)
df

,name
0,aisles
1,departments
2,order_products__prior
3,orders
4,products


## 1) aisles 테이블 조회

*   aisle_id: aisle identifier
*   aisle: the name of the aisle

In [13]:
#테이블 리스트 확인
sql = f"""
SELECT * FROM aisles;
"""
df = pd.read_sql(sql, conn, index_col=None)
df.head(10)

,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation
5,6,other
6,7,packaged meat
7,8,bakery desserts
8,9,pasta sauce
9,10,kitchen supplies


In [14]:
#테이블 리스트 확인
# 이걸 통해서 aisle_id는 각각 다 다르다는 것을 알 수 있지!

sql = f"""
SELECT count(*), count(distinct aisle_id) FROM aisles;
"""
df = pd.read_sql(sql, conn, index_col=None)
df.head(10)

,count(*),count(distinct aisle_id)
0,134,134


## 2) departments 테이블 조회

*   department_id: department identifier
*   department: the name of the department

In [17]:
#departments 테이블 조회
sql = f"""
SELECT * FROM departments;
"""
df = pd.read_sql(sql, conn, index_col=None)
df.head(10)

,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol
5,6,international
6,7,beverages
7,8,pets
8,9,dry goods pasta
9,10,bulk


In [18]:
#departments 테이블 uniq 필드 확인
sql = f"""
SELECT count(*), count(distinct department_id) FROM departments;
"""
df = pd.read_sql(sql, conn, index_col=None)
df.head(10)

,count(*),count(distinct department_id)
0,21,21


## 3) order_products__prior 테이블 조회

*   order_id: foreign key
*   product_id: foreign key
*   add_to_cart_order: order in which each product was added to cart
*   reordered: 1 if this product has been ordered by this user in the past, 0 otherwise

In [19]:
# order_products__prior 테이블 조회
sql = f"""
SELECT * FROM order_products__prior;
"""
df = pd.read_sql(sql, conn, index_col=None)
df.head(10)

,order_id,product_id,add_to_cart_order,reordered
0,2581,40174,1,0
1,2581,17461,2,0
2,2581,19731,3,1
3,5880,24838,1,1
4,5880,24852,2,1
5,5880,16797,3,1
6,5880,5456,4,0
7,5880,25199,5,0
8,6030,280,1,0
9,7374,25146,1,0


In [21]:
# order_products__prior 테이블 uniq 필드 확인
# union을 하면 행이 하나씩 추가됨

sql = f"""
-- uniq order_id,	product_id count
SELECT count(*) as cnt
  FROM (
SELECT distinct order_id,	product_id
  FROM order_products__prior
     ) A

-- uniq order_id count
union all
SELECT count(distinct order_id) as cnt
  FROM order_products__prior

-- all count
union all
SELECT count(*) as cnt
  FROM order_products__prior
;
"""
df = pd.read_sql(sql, conn, index_col=None)
df.head(10)

,cnt
0,32290
1,3214


## 4) orders 테이블 조회

*   order_id: order identifier
*   user_id: customer identifier
*   eval_set: which evaluation set this order belongs in (see SET described below)
*   order_number: the order sequence number for this user (1 = first, n = nth)
*   order_dow: the day of the week the order was placed on
*   order_hour_of_day: the hour of the day the order was placed on
*   days_since_prior: days since the last order, capped at 30 (with NAs for *   order_number = 1)

In [22]:
# orders 테이블 조회
sql = f"""
SELECT * FROM orders;
"""
df = pd.read_sql(sql, conn, index_col=None)
df.head(10)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2352815,185282,prior,5,2,9,6
1,1414622,41949,prior,9,0,16,6
2,2534064,24631,prior,66,1,13,4
3,338572,95289,prior,4,2,14,18
4,2578618,170417,prior,5,1,13,14
5,1756647,127210,prior,31,3,18,9
6,2636806,26473,prior,24,2,17,17
7,681993,89155,prior,17,1,15,4
8,826339,67750,prior,7,3,14,17
9,57285,89307,prior,48,1,22,5


In [23]:
# orders 테이블 uniq 필드 확인
sql = f"""
-- uniq order_id,	user_id count
SELECT count(*) as cnt
  FROM (
SELECT distinct order_id,	user_id
  FROM orders
     ) A

-- uniq order_id count
union all
SELECT count(distinct order_id) as cnt
  FROM orders

-- all count
union all
SELECT count(*) as cnt
  FROM orders
;
"""
df = pd.read_sql(sql, conn, index_col=None)
df.head(10)

,cnt
0,3220
1,3220
2,3220


## 5) products 테이블 조회

*   product_id: product identifier
*   product_name: name of the product
*   aisle_id: foreign key
*   department_id: foreign key

In [24]:
#products 테이블 조회
sql = f"""
SELECT * FROM products;
"""
df = pd.read_sql(sql, conn, index_col=None)
df.head(10)

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,10,Sparkling Orange Juice & Prickly Pear Beverage,115,7
2,11,Peach Mango Juice,31,7
3,23,Organic Turkey Burgers,49,12
4,25,Salted Caramel Lean Protein & Fiber Bar,3,19
5,29,Fresh Cut Golden Sweet No Salt Added Whole Ker...,81,15
6,32,Nacho Cheese White Bean Chips,107,19
7,34,Peanut Butter Cereal,121,14
8,35,Italian Herb Porcini Mushrooms Chicken Sausage,106,12
9,37,Noodle Soup Mix With Chicken Broth,69,15


In [25]:
# products 테이블 uniq 필드 확인
sql = f"""
SELECT count(*), count(distinct product_id) FROM products;
"""
df = pd.read_sql(sql, conn, index_col=None)
df.head(10)

,count(*),count(distinct product_id)
0,9288,9288


#2.구매 지표 추출

## 1) 전체 주문 건수 (사전녹화)

*   데이터 중복 확인 : 주문 건수를 계산하기 위하여 데이터의 중복 여부를 확인함





*   만약 ORDERS라는 테이블에 주문 번호(ORDER_ID)가 중복되어 존재한다면 실제 주문 건수보다 더 큰 값이 계산됨
*   count(distinct var)로 중복없이 계산함

In [26]:
sql = f"""
SELECT COUNT(DISTINCT ORDER_ID) F
FROM ORDERS;
"""
sql_result(sql,'2_1전체주문건수')


SELECT COUNT(DISTINCT ORDER_ID) F
FROM ORDERS;

      F
0  3220
/content/drive/MyDrive/result/6w/2_1전체주문건수.csv


## 2) 구매자 수 (사전녹화)

*   만약 ORDERS라는 테이블에 고객 번호(USER_ID)가 중복되어 존재한다면 실제 고객 수 보다 더 큰 값이 계산됨
*   count(distinct var)로 중복없이 계산함

In [27]:
sql = f"""
SELECT COUNT(DISTINCT USER_ID) BU
FROM ORDERS;
"""
sql_result(sql,'2_1구매자수')


SELECT COUNT(DISTINCT USER_ID) BU
FROM ORDERS;

     BU
0  3159
/content/drive/MyDrive/result/6w/2_1구매자수.csv


## 3) 상품별 주문 건수 (사전녹화)

*   상품별 주문 건수를 계산하려면, 먼저 상품명으로 데이터를 그룹핑하고 ODER_ID를 카운트해 집계함		

*   주문 번호(ORDER_ID)는 ORDER_PRODUCTS_PRIOR라는 테이블에 존재함
*   상품명(PRODUCT_NAME)은 PRODUCTS라는 테이블에 존재함

*   STEP1 : 분산된 정보를 함께 사용하려면 2개의 테이블을 결합(JOIN)해야 함

In [32]:
#2개의 테이블에 공통으로 존재하는 product_id로 2개의 테이블을 JOIN함
# A에 order_products_prior, B에 products를 두고 왼쪽으로 join함
# 두 개 테이블이 겹치는 부분인 product_id를 기준으로...!
# A B 뒤집으면 안되긴 하네 ㅎㅅㅎ

sql = f"""
SELECT *
  FROM ORDER_PRODUCTS__PRIOR A --주문번호 데이터
  LEFT JOIN PRODUCTS B --상품명 데이터
    ON A.PRODUCT_ID = B.PRODUCT_ID --Join Key
;
"""
sql_result(sql,'2_3_1상품별주문건수')


SELECT *
  FROM ORDER_PRODUCTS__PRIOR A --주문번호 데이터
  LEFT JOIN PRODUCTS B --상품명 데이터
    ON A.PRODUCT_ID = B.PRODUCT_ID --Join Key
;

   order_id  product_id  ...  aisle_id  department_id
0      2581       40174  ...        84             16
1      2581       17461  ...        35             12
2      2581       19731  ...        31              7
3      5880       24838  ...        91             16
4      5880       24852  ...        24              4
5      5880       16797  ...        24              4
6      5880        5456  ...        96             20
7      5880       25199  ...         1             20
8      6030         280  ...        73             11
9      7374       25146  ...        31              7

[10 rows x 8 columns]
/content/drive/MyDrive/result/6w/2_3_1상품별주문건수.csv


*   JOIN  결과 order_products_prior의 product_id에 해당하는 product_name, aisle_id, department_id가 결합됨
		

*   STEP2 : 결합한 결과에서 PRODUCT_NAME으로 데이터를 그룹핑하고, ORDER_ID를 카운트 함

In [33]:
# PRODUCT_NAME으로 그룹핑하고 카운트 할 때 주문번호 중복을 제거해야 함
sql = f"""
SELECT B.PRODUCT_NAME
     , COUNT(DISTINCT A.ORDER_ID) F -- 주문번호 중복 제거
  FROM ORDER_PRODUCTS__PRIOR A
  LEFT JOIN PRODUCTS B
    ON A.PRODUCT_ID = B.PRODUCT_ID
 GROUP BY 1
;
"""
sql_result(sql,'2_3_2상품별주문건수')


SELECT B.PRODUCT_NAME
     , COUNT(DISTINCT A.ORDER_ID) F -- 주문번호 중복 제거
  FROM ORDER_PRODUCTS__PRIOR A
  LEFT JOIN PRODUCTS B
    ON A.PRODUCT_ID = B.PRODUCT_ID
 GROUP BY 1
;

                               product_name  F
0                         #2 Coffee Filters  1
1        0% Fat Black Cherry Greek Yogurt y  1
2             0% Fat Blueberry Greek Yogurt  1
3                  0% Fat Free Organic Milk  4
4       0% Fat Organic Greek Vanilla Yogurt  1
5           0% Fat Superfruits Greek Yogurt  1
6                  0% Greek Strained Yogurt  6
7  0% Greek, Blueberry on the Bottom Yogurt  2
8                           1 % Lowfat Milk  1
9               1 Apple + 1 Mango Fruit Bar  1
/content/drive/MyDrive/result/6w/2_3_2상품별주문건수.csv


## 4) 카트에 가장 먼저 넣는 상품 10개 (사전녹화)

*우리는 쇼핑몰에서 물건을 구입할 때, 장바구니 담기라는 기능을 자주 사용한다. 가장 필요한 상품을 먼저 검색하고 장바구니에 담는 경향이 있다. 그렇다면 장바구니에 먼저 담기는 상품들에는 무엇이 있을까?*
		

*   ORDER_PRODUCTS_PRIOR테이블을 보면 ADD_TO_CART_ORDER라는 칼럼이 존재함
*   ADD_TO_CART_ORDER는 상품이 몇 번째로 장바구니에 담겼는지를 의미함

*예를 들어, 주문 번호(ORDER_ID) 2581에서 40174라는 상품은 첫 번째로 장바구니에 담겼고, 19731이라는 상품은 세 번째로 장바구니에 담겼다.*
		

In [34]:
# order_products__prior 2581 주문번호 조회
sql = f"""
SELECT * FROM order_products__prior WHERE order_id = 2581;
"""
df = pd.read_sql(sql, conn, index_col=None)
df.head(10)

,order_id,product_id,add_to_cart_order,reordered
0,2581,40174,1,0
1,2581,17461,2,0
2,2581,19731,3,1


*   mineral water라는 상품이 A라는 주문 건에서 두 번째로 담기고, B라는 주문 건에서 첫 번째로 담기는 경우에 mineral water가 첫 번째로 카트에 담긴 건수는 1건이 됨

<center>

|주문번호|상품|장바구니순번|
|-----|-----|-----|
|A|mineral water|2|
|B|mineral water|1|

</center>

*   STEP1 : ORDER_PRODUCTS_PRIOR의 PRODUCT_ID별로 가장 먼저 담긴 경우에는 1을 출력하고 그 외 0을 출력하는 칼럼을 생성함
*   CASE WHEN ADD_TO_CART_ORDER=1 THEN 1 ELSE 0 END

In [35]:
sql = f"""
SELECT PRODUCT_ID
     , CASE WHEN ADD_TO_CART_ORDER = 1 THEN 1 ELSE 0 END F_1ST
  FROM ORDER_PRODUCTS__PRIOR
;
"""
sql_result(sql,'2_4_1카트에가장먼저넣는상품10개')


SELECT PRODUCT_ID
     , CASE WHEN ADD_TO_CART_ORDER = 1 THEN 1 ELSE 0 END F_1ST
  FROM ORDER_PRODUCTS__PRIOR
;

   product_id  F_1ST
0       40174      1
1       17461      0
2       19731      0
3       24838      1
4       24852      0
5       16797      0
6        5456      0
7       25199      0
8         280      1
9       25146      1
/content/drive/MyDrive/result/6w/2_4_1카트에가장먼저넣는상품10개.csv


*   STEP2 : 상품 번호(PRODUCT_ID)로 데이터를 그룹핑하고 F_1ST 칼럼을 합 함

In [36]:
sql = f"""
SELECT PRODUCT_ID
     , SUM(CASE WHEN ADD_TO_CART_ORDER = 1 THEN 1 ELSE 0 END) F_1ST
  FROM ORDER_PRODUCTS__PRIOR
 GROUP BY 1
;
"""
sql_result(sql,'2_4_2카트에가장먼저넣는상품10개')


SELECT PRODUCT_ID
     , SUM(CASE WHEN ADD_TO_CART_ORDER = 1 THEN 1 ELSE 0 END) F_1ST
  FROM ORDER_PRODUCTS__PRIOR
 GROUP BY 1
;

   product_id  F_1ST
0           1      0
1          10      0
2          11      0
3          23      0
4          25      0
5          29      0
6          32      0
7          34      0
8          35      0
9          37      0
/content/drive/MyDrive/result/6w/2_4_2카트에가장먼저넣는상품10개.csv


*   STEP3 : F_1ST(장바구니에 가장 먼저 담긴 건수)로 데이터에 순서를 부여함
*   ROW_NUMBER(), RANK()

In [37]:
# rank는 똑같은 거 나오면 그냥 똑같은 순위로
# row_number는 똑같은 거 나와도 order되는 순서대로 순위 매김

sql = f"""
-- 상품별 첫 번째로 장바구니에 담은 횟수 기준 순번
SELECT *
     , ROW_NUMBER() OVER(ORDER BY F_1ST DESC) RNK

-- 상품별 첫 번째로 장바구니에 담은 횟수
  FROM (    
SELECT PRODUCT_ID
     , SUM(CASE WHEN ADD_TO_CART_ORDER = 1 THEN 1 ELSE 0 END) F_1ST
  FROM ORDER_PRODUCTS__PRIOR
 GROUP BY 1
     ) A
;
"""
sql_result(sql,'2_4_3카트에가장먼저넣는상품10개')


-- 상품별 첫 번째로 장바구니에 담은 횟수 기준 순번
SELECT *
     , ROW_NUMBER() OVER(ORDER BY F_1ST DESC) RNK

-- 상품별 첫 번째로 장바구니에 담은 횟수
  FROM (    
SELECT PRODUCT_ID
     , SUM(CASE WHEN ADD_TO_CART_ORDER = 1 THEN 1 ELSE 0 END) F_1ST
  FROM ORDER_PRODUCTS__PRIOR
 GROUP BY 1
     ) A
;

   PRODUCT_ID  F_1ST  RNK
0       24852    117    1
1       13176     62    2
2       27845     37    3
3       21137     31    4
4       21903     27    5
5       47209     24    6
6       19660     18    7
7       16797     17    8
8        5785     16    9
9       12341     16   10
/content/drive/MyDrive/result/6w/2_4_3카트에가장먼저넣는상품10개.csv


*   STEP4 : 1-10위의 상품 번호만 출력하기 위해서 WHERE 절에 조건을 추가해 RNK가 1-10 사이인 데이터만 출력함
*   이때 RNK는 SELECT문에서 새롭게 생성한 칼럼이므로 WHERE 절에서 바로 사용할 수가 없음
*   따라서 위의 쿼리를 SUBQUERY로 사용해 조건을 생성해야 함

In [38]:
sql = f"""
SELECT *

-- 상품별 첫 번째로 장바구니에 담은 횟수 기준 순번
  FROM (
SELECT *
     , ROW_NUMBER() OVER(ORDER BY F_1ST DESC) RNK

-- 상품별 첫 번째로 장바구니에 담은 횟수
  FROM (
SELECT PRODUCT_ID
     , SUM(CASE WHEN ADD_TO_CART_ORDER = 1 THEN 1 ELSE 0 END) F_1ST
  FROM ORDER_PRODUCTS__PRIOR
 GROUP BY 1
     ) A

     ) BASE

-- 상품별 첫 번째로 장바구니에 담은 횟수 기준 순번 1~10번
WHERE RNK BETWEEN 1 AND 10;

"""
sql_result(sql,'2_4_4카트에가장먼저넣는상품10개')


SELECT *

-- 상품별 첫 번째로 장바구니에 담은 횟수 기준 순번
  FROM (
SELECT *
     , ROW_NUMBER() OVER(ORDER BY F_1ST DESC) RNK

-- 상품별 첫 번째로 장바구니에 담은 횟수
  FROM (
SELECT PRODUCT_ID
     , SUM(CASE WHEN ADD_TO_CART_ORDER = 1 THEN 1 ELSE 0 END) F_1ST
  FROM ORDER_PRODUCTS__PRIOR
 GROUP BY 1
     ) A

     ) BASE

-- 상품별 첫 번째로 장바구니에 담은 횟수 기준 순번 1~10번
WHERE RNK BETWEEN 1 AND 10;


   PRODUCT_ID  F_1ST  RNK
0       24852    117    1
1       13176     62    2
2       27845     37    3
3       21137     31    4
4       21903     27    5
5       47209     24    6
6       19660     18    7
7       16797     17    8
8        5785     16    9
9       12341     16   10
/content/drive/MyDrive/result/6w/2_4_4카트에가장먼저넣는상품10개.csv


*   ORDER BY를 이용해 간단히 상위 10개의 데이터 호출하는 법

In [39]:
# order by N : N번째 열을 기준으로 정렬하삼

sql = f"""
SELECT PRODUCT_ID
     , SUM(CASE WHEN ADD_TO_CART_ORDER = 1 THEN 1 ELSE 0 END) F_1ST
  FROM ORDER_PRODUCTS__PRIOR
 GROUP BY 1
 ORDER BY 2 DESC
 LIMIT 10
;
"""
sql_result(sql,'2_4_5카트에가장먼저넣는상품10개')


SELECT PRODUCT_ID
     , SUM(CASE WHEN ADD_TO_CART_ORDER = 1 THEN 1 ELSE 0 END) F_1ST
  FROM ORDER_PRODUCTS__PRIOR
 GROUP BY 1
 ORDER BY 2 DESC
 LIMIT 10
;

   product_id  F_1ST
0       24852    117
1       13176     62
2       27845     37
3       21137     31
4       21903     27
5       47209     24
6       19660     18
7       16797     17
8       12341     16
9        5785     16
/content/drive/MyDrive/result/6w/2_4_5카트에가장먼저넣는상품10개.csv


## 5) 시간별 주문 건수 (사전녹화)

*   ORDERS의 ORDER_HOUR_OF_DAY로 그룹핑한 뒤, ORDER_ID를 카운트 함
*   ORDER_ID에 중복이 있을 수 있어 DISTINCT를 추가해 중복을 제거함 (예제에는 중복이 없음)

In [40]:
sql = f"""
SELECT ORDER_HOUR_OF_DAY
     , COUNT(DISTINCT ORDER_ID) F
  FROM ORDERS
 GROUP BY 1
 ORDER BY 1;
"""
sql_result(sql,'2_5_1시간별주문건수')


SELECT ORDER_HOUR_OF_DAY
     , COUNT(DISTINCT ORDER_ID) F
  FROM ORDERS
 GROUP BY 1
 ORDER BY 1;

   order_hour_of_day    F
0                  0   21
1                  1   12
2                  2    3
3                  3    7
4                  4    2
5                  5    4
6                  6   33
7                  7   95
8                  8  163
9                  9  239
/content/drive/MyDrive/result/6w/2_5_1시간별주문건수.csv


## 6) 첫 구매 후 다음 구매까지 걸린 평균 일수

*   ORDERS 테이블에서 DAYS_SINCE_PRIOR_ORDER는 이전 주문이 이루어진 지 며칠 뒤에 구매가 이루어졌는지를 나타내는 값 임
*   주문 번호의 ORDERS_NUMBER가 2인(유저의 2번째 주문 건) DAYS_SINCE_PRIOR_ORDERS는 첫 구매 후 다음 구매까지 걸린 기간이 됨
*   해당 기간의 평균을 구하면 첫 구매 후 다음 구매까지 걸린 평균 일수를 계산할 수 있음

In [41]:
# order number가 2인 것 빼내기 위함
sql = f"""
SELECT AVG(DAYS_SINCE_PRIOR_ORDER) AVG_RECENCY
  FROM ORDERS
 WHERE ORDER_NUMBER = 2 
;
"""
sql_result(sql,'2_6_1첫구매후다음구매까지걸린평균일수')


SELECT AVG(DAYS_SINCE_PRIOR_ORDER) AVG_RECENCY
  FROM ORDERS
 WHERE ORDER_NUMBER = 2 
;

   AVG_RECENCY
0    13.651786
/content/drive/MyDrive/result/6w/2_6_1첫구매후다음구매까지걸린평균일수.csv


## 7) 주문 건당 평균 구매 상품 수(UPT, Unit Per Transaction)

*   PRODUCT_ID를 카운트해 상품 개수를 계산하고, 이를 주문 건수로 나누어 주문 1건에 평균적으로 몇 개의 상품을 구매하는지 파악할 수 있음

In [42]:
# 건 상품 수
# 주문 건당 평균 구매 상품 수. 주문 한 개에 몇 개의 상품이 달려있는지. basket 안에 몇 개의 상품이 들어있는가.
# product id를 count하면 상품 개수 구할 수 있음. 이거를 주문 건수로 나누면 한 건에 평균 몇 개를 구매하는지 파악 가능
# 전략이나 서비스 정책 세우기 위해... 몇 개를 담는지 확인해야함!
# COUNT는 집계 함수. DISTINCT를 사용함으로써 unique함 살릴 수 있음.
# 0.0을 왜 더하는가? : sqlite3는 기업에서 쓰는 용도가 아님... 원래 휴대폰같은 곳에서 데이터 저장하고 관리하는 db...라서 숫자같은 걸 버림함...
#                       data type을 변경해주는 방식... 약간 int형에서 float형으로 바꿔주는 거지.

sql = f"""
SELECT (COUNT(PRODUCT_ID) + 0.0)/(COUNT(DISTINCT ORDER_ID) + 0.0) UPT
  FROM ORDER_PRODUCTS__PRIOR
;
"""
sql_result(sql,'2_7_1주문건당평균구매상품수')


SELECT (COUNT(PRODUCT_ID) + 0.0)/(COUNT(DISTINCT ORDER_ID) + 0.0) UPT
  FROM ORDER_PRODUCTS__PRIOR
;

         UPT
0  10.046671
/content/drive/MyDrive/result/6w/2_7_1주문건당평균구매상품수.csv


## 8) 인당 평균 주문 건수

*   거래당 구매 상품 수를 구했듯이, 전체 주문 건수를 구매자 수로 나누어 인당 평균 주문 건수를 계산할 수 있음
*   PRODUCT_ID를 카운트해 상품 개수를 계산하고, 이를 주문 건수로 나누어 주문 1건에 평균적으로 몇 개의 상품을 구매하는지 파악할 수 있음

In [43]:
# order id와 user id에 distinct 걸었음... 걸어주는 게 좋은 습관이긴 해... 중복 안 된다는 보장이 없음
# 근데 distinct 안 걸어도 결과가 동일하게 나오긴 함.

sql = f"""
SELECT (COUNT(DISTINCT ORDER_ID)+0.0)/(COUNT(DISTINCT USER_ID)+0.0) AVG_F
  FROM ORDERS;
"""
sql_result(sql,'2_8_1인당평균주문건수')


SELECT (COUNT(DISTINCT ORDER_ID)+0.0)/(COUNT(DISTINCT USER_ID)+0.0) AVG_F
  FROM ORDERS;

     AVG_F
0  1.01931
/content/drive/MyDrive/result/6w/2_8_1인당평균주문건수.csv


## 9) 재구매율이 가장 높은 상품 10개

*   상품별로 재구매율을 계산한 뒤, 재구매율을 기준으로 순위를 계산함
*   순위를 기준으로 원하는 순위값(1~10)으로 조건을 생성함

### *STEP1 : 상품별 재구매율 계산

*   상품 번호로 데이터를 그룹핑하고, 재구매 수(SUM(CASE WHEN REORDERED=1 THEN 1 ELSE 0 END)를 전체 구매 수(COUNT(*))로 나누어 재구매율을 계산함

In [44]:
# 재구매인지 아닌지 알기 위해 reordered 사용함. 재구매의 경우 reordered.
# case when이 약간 if같은 그런 느낌인가봐... 재주문이면 1로 바꾸고, 아니면 0으로.
# 1로 바꾼 이유 : 더해주기 위해서...
# 재구매건수 / 전체 건수 == 재구매율
# 주문 전체 건수가 작으면 작을수록 재구매율이 높게 나올 수 있음.

sql = f"""
SELECT PRODUCT_ID
-- 재구매율 계산
     , SUM(CASE WHEN REORDERED =1 THEN 1 ELSE 0 END + 0.0)/COUNT(*) RET_RATIO
  FROM ORDER_PRODUCTS__PRIOR
 GROUP BY 1
;
"""
sql_result(sql,'2_9_1상품별재구매율계산')


SELECT PRODUCT_ID
-- 재구매율 계산
     , SUM(CASE WHEN REORDERED =1 THEN 1 ELSE 0 END + 0.0)/COUNT(*) RET_RATIO
  FROM ORDER_PRODUCTS__PRIOR
 GROUP BY 1
;

   product_id  RET_RATIO
0           1   0.333333
1          10   0.500000
2          11   0.000000
3          23   1.000000
4          25   1.000000
5          29   1.000000
6          32   1.000000
7          34   0.333333
8          35   0.000000
9          37   1.000000
/content/drive/MyDrive/result/6w/2_9_1상품별재구매율계산.csv


### *STEP2 : 재구매율로 랭크(순위)열 생성하기

*   상품별 재구매율로 순위를 생성함
*   재구매율(RET_RATIO)로 순위를 매기려면, 앞의 결과를 SUBQUERY로 생성하고, SELECT문에서 ROW_NUMBER/RANK 중 하나를 선택해 순위 열을 생성함

In [45]:
# rank number는 순위가 같으면 그냥 같은 값으로 넣어주는데
# row number는 순위 같던 말던 그냥 index 형태로 숫자 와르륵 넣어줌
# 재구매율이 높은 item을 1등으로 하기 위해서 order by(정렬), descending(내림차순) 적용

sql = f"""
SELECT *
-- 순위 계산
     , ROW_NUMBER() OVER(ORDER BY RET_RATIO DESC) RNK   /* subquery로 순위 계산 */
  FROM (
SELECT PRODUCT_ID
     , SUM(CASE WHEN REORDERED =1 THEN 1 ELSE 0 END + 0.0)/COUNT(*) RET_RATIO
  FROM ORDER_PRODUCTS__PRIOR
GROUP BY 1
    ) A
;
"""
sql_result(sql,'2_9_2재구매율로랭크(순위)열생성하기')


SELECT *
-- 순위 계산
     , ROW_NUMBER() OVER(ORDER BY RET_RATIO DESC) RNK   /* subquery로 순위 계산 */
  FROM (
SELECT PRODUCT_ID
     , SUM(CASE WHEN REORDERED =1 THEN 1 ELSE 0 END + 0.0)/COUNT(*) RET_RATIO
  FROM ORDER_PRODUCTS__PRIOR
GROUP BY 1
    ) A
;

   PRODUCT_ID  RET_RATIO  RNK
0          23        1.0    1
1          25        1.0    2
2          29        1.0    3
3          32        1.0    4
4          37        1.0    5
5          40        1.0    6
6          47        1.0    7
7         101        1.0    8
8         106        1.0    9
9         128        1.0   10
/content/drive/MyDrive/result/6w/2_9_2재구매율로랭크(순위)열생성하기.csv


### *STEP3 : TOP 10(재구매율)상품 추출 

*   생성한 순위열의 값이 1~10 사이인 데이터를 조회하려면, 앞의 결과를 SUBQUERY로 생성하고, WHERE절에서 RNK BETWEEN 1 AND 10을 선택함

In [46]:
# 재구매율이 높은 top 10개 뽑기!
# rnk(rank) 만들어둔 거에서 1에서 10까지 선택하기. (한 번 더 subquery 씌워야 함.)
# sql 사용하는 이유 : 큰 데이터 다루기 위해...

sql = f"""
SELECT *
  FROM (
SELECT *
     , ROW_NUMBER() OVER(ORDER BY RET_RATIO DESC) RNK
  FROM (
SELECT PRODUCT_ID
     , SUM(CASE WHEN REORDERED =1 THEN 1 ELSE 0 END + 0.0)/COUNT(*) RET_RATIO
  FROM ORDER_PRODUCTS__PRIOR
 GROUP BY 1
     ) A
     ) A -- WHERE절 입력을 위해 서브쿼리 사용
 WHERE RNK BETWEEN 1 AND 10 -- RNK 1~10 선택
;
"""
sql_result(sql,'2_9_3TOP10(재구매율)상품추출')


SELECT *
  FROM (
SELECT *
     , ROW_NUMBER() OVER(ORDER BY RET_RATIO DESC) RNK
  FROM (
SELECT PRODUCT_ID
     , SUM(CASE WHEN REORDERED =1 THEN 1 ELSE 0 END + 0.0)/COUNT(*) RET_RATIO
  FROM ORDER_PRODUCTS__PRIOR
 GROUP BY 1
     ) A
     ) A -- WHERE절 입력을 위해 서브쿼리 사용
 WHERE RNK BETWEEN 1 AND 10 -- RNK 1~10 선택
;

   PRODUCT_ID  RET_RATIO  RNK
0          23        1.0    1
1          25        1.0    2
2          29        1.0    3
3          32        1.0    4
4          37        1.0    5
5          40        1.0    6
6          47        1.0    7
7         101        1.0    8
8         106        1.0    9
9         128        1.0   10
/content/drive/MyDrive/result/6w/2_9_3TOP10(재구매율)상품추출.csv


## 10) Department별 재구매율이 가장 높은 상품 10개

*앞에서 상품별로 재구매율을 구한 뒤 재구매율이 가장 높은 상품 10개를 추출했다. Department별로 재구매율이 높은 상위 10개 상품을 추출하려면 어떻게 해야 할까?*
		

*   Department별로 재구매율이 높은 상품을 추출하려면, Department별, 상품별 재구매율을 계산함
*   Department로 Partition을 생성한 뒤 순위를 계산하면 됨
*   최종적으로 원하는 조건(RNK:1~10)을 생성하면, Department별 재구매율 상위 10개의 데이터를 추출할 수 있음
		

*위의 예제에서 추가된 내용은 ORDERS_PRODUCT_PRIOR에 PRODUCTS 테이블을 조인해 DEPARTMENT를 가져와야 한다는 점과 DEPARTMENT로 PARTITION을 생성해 순위를 매겨야 한다는 점이다.*

In [47]:
# 앞이랑 똑같은데 다른 점이 있다면 department별로 group by 시켜주는 거!
# department 바로 연결 안 돼서 product table이랑 연결함... 그 안에서의 departments까지도 접근 시도
# 와 뭔소린지 전혀 모르겠음... 큰일났네...^^
# groupby를 department별로 한 번, product별로 한 번 하면 department별 재구매율의 rank 나옴
# product id는 department의 id에 대해 N:1의 관계. (정장이 가전이랑 의류 양쪽에 들어갈 순 없잖아.)
# 두 번에 걸쳐서 group by해줘도 되긴 한데 한 번에 하는 게 낫지 않겠어?.? (예제에서도 붙여주고 있음)
# sqlite는 굉장히 high level language,,, 한 눈에 알아볼 수 있잖아... 함수가 그 의미를 다 바로 담고 있잖아
# partition by department : department별로 나눈다!
# descend 걸면 큰 값에서 점점 내려가는 걸로...
# rank를 subquery로 감싸줘서 뽑음

sql = f"""
SELECT *
  FROM (
SELECT *

-- PARTITION BY DEPARTMENT 부서별 순위 계산
     , ROW_NUMBER() OVER (PARTITION BY DEPARTMENT ORDER BY RET_RATIO DESC) RNK

  FROM (
SELECT C.DEPARTMENT --Department 추가
     , A.PRODUCT_ID
     , SUM(CASE WHEN REORDERED =1 THEN 1 ELSE 0 END + 0.0)/COUNT(*) RET_RATIO
  FROM ORDER_PRODUCTS__PRIOR A

--departments를 추가하기 위해서 products 테이블 추가
  LEFT JOIN PRODUCTS B ON A.PRODUCT_ID = B.PRODUCT_ID

--departments 테이블 추가
  LEFT JOIN DEPARTMENTS C ON B.DEPARTMENT_ID = C.DEPARTMENT_ID

 GROUP BY 1,2
     ) A

     ) A -- WHERE절 입력을 위해 서브쿼리 사용
 WHERE RNK BETWEEN 1 AND 10
;
"""
sql_result(sql,'2_10_1Department별상품10개')


SELECT *
  FROM (
SELECT *

-- PARTITION BY DEPARTMENT 부서별 순위 계산
     , ROW_NUMBER() OVER (PARTITION BY DEPARTMENT ORDER BY RET_RATIO DESC) RNK

  FROM (
SELECT C.DEPARTMENT --Department 추가
     , A.PRODUCT_ID
     , SUM(CASE WHEN REORDERED =1 THEN 1 ELSE 0 END + 0.0)/COUNT(*) RET_RATIO
  FROM ORDER_PRODUCTS__PRIOR A

--departments를 추가하기 위해서 products 테이블 추가
  LEFT JOIN PRODUCTS B ON A.PRODUCT_ID = B.PRODUCT_ID

--departments 테이블 추가
  LEFT JOIN DEPARTMENTS C ON B.DEPARTMENT_ID = C.DEPARTMENT_ID

 GROUP BY 1,2
     ) A

     ) A -- WHERE절 입력을 위해 서브쿼리 사용
 WHERE RNK BETWEEN 1 AND 10
;

  DEPARTMENT  PRODUCT_ID  RET_RATIO  RNK
0    alcohol        1168        1.0    1
1    alcohol        2054        1.0    2
2    alcohol        3828        1.0    3
3    alcohol        5146        1.0    4
4    alcohol        5470        1.0    5
5    alcohol        6263        1.0    6
6    alcohol       11541        1.0    7
7    alcohol       13303        1.0    8
8    alcohol       14974        1.0    9
